#### Connect Jupyter Notebook to the database file

In [9]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

#### Table details

In [10]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


#### Row preview

In [11]:
%%sql
select * from facts limit 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


#### Min/max population, min/max population growth

In [12]:
%%sql
select min(population) min_population, max(population) max_population, min(population_growth) min_population_growth, max(population_growth) max_population_growth
from facts 

Done.


min_population,max_population,min_population_growth,max_population_growth
0,7256490011,0.0,4.02


#### Lowest population

In [109]:
%%sql
select name country, population 
from facts 
where population = (select min(population) from facts)
or population = (select min(population) from facts where name != "Antarctica")
order by population

Done.


country,population
Antarctica,0
Pitcairn Islands,48


#### Highest population

In [102]:
%%sql
select name country, population
from facts 
where population = (select max(population) from facts where name != "World")

Done.


country,population
China,1367485388


#### Avgerage population, average area

In [15]:
%%sql
select avg(population) avg_populatin, avg(area) avg_area from facts

Done.


avg_populatin,avg_area
62094928.32231405,555093.546184739


####  Countries with higher than average populatin and lower than average, ordered by highest density

In [145]:
%%sql
select
name country, population, area, population / area as density
from facts
where population > (
    select avg(population)
    from facts
) 
    and area < (
        select avg(area)
        from facts
)
order by density desc

Done.


country,population,area,density
Bangladesh,168957745,148460,1138
Philippines,100998376,300000,336
Japan,126919659,377915,335
Vietnam,94348835,331210,284
United Kingdom,64088222,243610,263
Germany,80854408,357022,226
Thailand,67976405,513120,132


#### Lowest density

In [180]:
%%sql
select name country, population / area as density
from facts
where density != "None"
order by density
limit 7

Done.


country,density
Greenland,0
Svalbard,0
Falkland Islands (Islas Malvinas),0
Mongolia,1
Pitcairn Islands,1
Australia,2
Namibia,2


#### Average land area, average water area

In [17]:
%%sql
select avg(area_land) avg_area_land, avg(area_water) avg_area_water from facts

Done.


avg_area_land,avg_area_water
522702.57723577233,19067.59259259259


#### Water to land ratio

In [18]:
%%sql
select avg(cast(area_water as float) / cast(area_land as float)) ratio_water_land from facts

Done.


ratio_water_land
3.803454624004138


#### Countries with highest ratios of water to land

In [179]:
%%sql
select name country, cast(area_water as float) / cast(area_land as float) ratio_water_land 
from facts
order by ratio_water_land desc
limit 5

Done.


country,ratio_water_land
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623


#### Countries with more water than land area

In [178]:
%%sql
select name country, area, area_water, area_land, area_water - area_land water_land_diff 
from facts
where area_water > area_land


Done.


country,area,area_water,area_land,water_land_diff
British Indian Ocean Territory,54400,54340,60,54280
Virgin Islands,1910,1564,346,1218


#### Highest asbsolute population growth

In [177]:
%%sql
select name country, area, population, population_growth, 
cast((population * population_growth) - population as int) absolute_growth,
birth_rate, death_rate, migration_rate
from facts 
where name != "World"
order by absolute_growth desc
limit 5

Done.


country,area,population,population_growth,absolute_growth,birth_rate,death_rate,migration_rate
India,3287263,1251695584,1.22,275373028,19.55,7.32,0.04
Nigeria,923768,181562056,2.45,263264981,37.64,12.9,0.22
Ethiopia,1104300,99465819,2.89,187990397,37.27,8.19,0.22
"Congo, Democratic Republic of the",2344858,79375136,2.45,115093947,34.88,10.07,0.27
Bangladesh,148460,168957745,1.6,101374647,21.14,5.61,0.46


#### Highest population growth rate

In [176]:
%%sql
select name country, max(population_growth) as max_pop_rate from facts

Done.


country,max_pop_rate
South Sudan,4.02


In [175]:
%%sql
select name country, population_growth as min_pop_rate from facts
where min_pop_rate != "None"
order by population_growth desc
limit 5

Done.


country,min_pop_rate
South Sudan,4.02
Malawi,3.32
Burundi,3.28
Niger,3.25
Uganda,3.24


#### Lowest absolute population growth

In [153]:
%%sql
select name, area, population, population_growth, 
cast((population * population_growth) - population as int) absolute_growth,
birth_rate, death_rate, migration_rate
from facts 
where absolute_growth not in ("World", "None")
order by absolute_growth asc
limit 5

Done.


name,area,population,population_growth,absolute_growth,birth_rate,death_rate,migration_rate
China,9596960,1367485388,0.45,-752116963,12.49,7.53,0.44
European Union,4324782,513949445,0.25,-385462083,10.2,10.2,2.5
Russia,17098242,142423773,0.04,-136726822,11.6,13.69,1.69
Japan,377915,126919659,0.16,-106612513,7.93,9.51,0.0
United States,9826675,321368864,0.78,-70701150,12.49,8.15,3.86


#### Lowest population growth rate

In [174]:
%%sql
select name country, population_growth as min_pop_rate from facts
where min_pop_rate != "None"
order by population_growth
limit 5

Done.


country,min_pop_rate
Holy See (Vatican City),0.0
Cocos (Keeling) Islands,0.0
Greenland,0.0
Pitcairn Islands,0.0
Greece,0.01


#### Countries where death rate exceeds birth rate

In [173]:
%%sql
select name country, population, death_rate, birth_rate, round(death_rate - birth_rate, 2) as diff_death_birth_rate from facts
where death_rate > birth_rate
order by diff_death_birth_rate desc
limit 5

Done.


country,population,death_rate,birth_rate,diff_death_birth_rate
Bulgaria,7186893,14.44,8.92,5.52
Serbia,7176794,13.66,9.08,4.58
Latvia,1986705,14.31,10.0,4.31
Lithuania,2884433,14.27,10.1,4.17
Ukraine,44429471,14.46,10.72,3.74
